In [ ]:
%pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

## トークナイザとモデルの読み込み

**submit model: https://huggingface.co/geniacllm/dMoE_8B_finetune_all_v6_epoch2_v0.1**

* huggingfaceのmodelにtokenizerは含まれている

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_id = "geniacllm/dMoE_8B_finetune_all_v6_epoch2_v0.1" #@param {type:"string"}
use_fast = False #@param {type:"boolean"}

torch_dtype = "auto" #@param ["auto", "float16", "float32"]
device_map = "auto" #@param {type:"string"}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=use_fast, torch_dtype=torch_dtype, device_map=device_map)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch_dtype, device_map=device_map)

tokenizer_config.json:   0%|          | 0.00/960 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/972k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.66M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/859 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
# fine tuneできる層の出力
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
print_trainable_parameters(model)

trainable params: 8955660288 || all params: 8955660288 || trainable%: 100.0


## 推論の設定

leaderboardは次の設定を強制されます see: https://github.com/matsuolab/ucllm_nedo_prod/tree/main/eval

```yaml
generator:
  do_sample: false
  num_beams: 1
  top_p: 1.0
  top_k: 0
  temperature: 0.1
  repetition_penalty: 1.0
```

In [ ]:
add_special_tokens = True #@param {type:"boolean"}

max_new_tokens = 50 #@param {type:"integer"}
repetition_penalty = 1.0 #@param {type:"number"}
do_sample = False #@param {type:"boolean"}
temperature = 0.1 #@param {type:"number"}

skip_special_tokens = False #@param {type:"boolean"}


## シングルターンの確認

議論ポイント

* ~~output_textにinput_textがそのまま含まれている？これは正常？~~
  * これはこのシートの実装の問題！ でした。修正済みです

In [ ]:
input_list = [
    "地球温暖化の主な原因となる物質は",
    "落とし物を拾って持ち帰るのは犯罪ですか？",
    "ポケモンとはなんですか？",
    "ポケモンとは",
    "My name is",
    "1+1=",
    "りんごが1個、バナナが2個あります。フルーツは何個ありますか？",
    "1皿にりんごが3個あります。この皿が3枚あるとき、りんごは何個ありますか？",
    "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?",
    "複雑な方程式のセットから、各方程式からすべての変数名を抽出します。結果をJSON文字列として返し、各方程式に一行を割り当ててください。\n1) y = (3/4)x^3 - e^(2x) + sin(pi*x) - sqrt(7)\n2) 2A - B/(3+C) * sum(N=1 to 5; ln(N)^2) = 5D*integral(a=0 to pi; cos(comb(N=1 to 10; N*a)))\n3) E = m(c^2) + gamma*(v/d)/(-(alpha/2) + sqrt(beta^2 + (alpha/2)^2))" + "方程式を並べ替えて、変数として 'a', 'b', 'c', 'd' などを使用してください。",
    "以下の株価の記録から、2022年の各月の最高および最低の終値を抽出します。結果をCSV文字列として返し、各月に一行を割り当ててください。\n日付,開始,高値,安値,終値,ボリューム\n2022-01-01,150.02,155.28,148.50,153.80,15678900\n2022-01-02,154.32,157.25,153.48,156.25,19874500\n2022-02-01,160.50,163.28,159.50,161.80,14326700\n2022-02-02,161.80,164.25,161.30,163.90,17689200\n2022-03-01,165.40,168.35,163.10,166.80,16253400\n2022-03-02,167.00,169.85,165.50,168.20,19568100"+"同じタスクをJSON形式で再度行い、回答内のすべての数値を最も近い整数に丸めてください。",
    '''<s>以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示:
与えられた文章から固有表現で書かれたターゲットの名前を抽出し、それに対する極性をpositive、neutral、negativeの中から選択して下さい。固有表現で書かれたターゲットの名前と、それに対する極性（positive、neutral、negativeのいずれか）のペアをスペース（ ）で区切って出力し、それ以外には何も含めないことを厳守してください。答えが複数の場合、改行で繋げてください。ただし、ターゲットは固有表現である市場、市況、会社/法人、グループ、会社内の部門、事業部、事業領域、製品、サービスの名称などを指すこととします。

### 入力:
文章：また、各種プロモーションを実施したことによりショッピングの取扱高は増加、アリペイやApple Payなどの決済メニューの多様化にも取り組みました

### 応答:'''
]

In [ ]:
for input_text in input_list:
    model.generation_config.pad_token_id = model.generation_config.eos_token_id
    inputs = tokenizer(input_text, return_tensors="pt", add_special_tokens=add_special_tokens).to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, repetition_penalty=repetition_penalty, do_sample=do_sample, temperature=temperature)
    # outputsの先頭のtensorがinputsと同じ部分までは表示しない
    filtered_outputs = outputs[0][len(inputs.input_ids[0]):]
    filtered_output_text = tokenizer.decode(filtered_outputs, skip_special_tokens=skip_special_tokens)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=skip_special_tokens)

    print("input_text: ", input_text)
    print("filtered_output_text: ", filtered_output_text)
    print("補足)output_text: ", output_text)
    print("---------------------------------\n")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


input_text:  地球温暖化の主な原因となる物質は
filtered_output_text:  、二酸化炭素(CO2)やメタン(CH4)、一酸化二窒素(N2O)、フロンガスなどがあります。これらの物質は、化石燃料の燃焼や森林伐採、農地開発、化石燃料の
補足)output_text:  <s>地球温暖化の主な原因となる物質は、二酸化炭素(CO2)やメタン(CH4)、一酸化二窒素(N2O)、フロンガスなどがあります。これらの物質は、化石燃料の燃焼や森林伐採、農地開発、化石燃料の
---------------------------------

input_text:  落とし物を拾って持ち帰るのは犯罪ですか？
filtered_output_text:   
10000</s>
補足)output_text:  <s> 落とし物を拾って持ち帰るのは犯罪ですか？
10000</s>
---------------------------------

input_text:  ポケモンとはなんですか？
filtered_output_text:   ポケモンとは、1996年に任天堂から発売されたゲームソフト『ポケットモンスター』シリーズに登場する架空の生物の総称です。</s>
補足)output_text:  <s> ポケモンとはなんですか？ポケモンとは、1996年に任天堂から発売されたゲームソフト『ポケットモンスター』シリーズに登場する架空の生物の総称です。</s>
---------------------------------

input_text:  ポケモンとは
filtered_output_text:   、1996年に任天堂から発売されたゲームソフト『ポケットモンスター』シリーズに登場する架空の生物の総称である。ゲームの他、アニメやカードゲームなどの関連商品も展開されている。</s>
補足)output_text:  <s> ポケモンとは、1996年に任天堂から発売されたゲームソフト『ポケットモンスター』シリーズに登場する架空の生物の総称である。ゲームの他、アニメやカードゲームなどの関連商品も展開されている。</s>
---------------------------------

input_text:  My name is
fi

## マルチターンの確認

議論ポイント

* ~~output_textにinput_textがそのまま含まれている？これは正常？（同上）~~
  * これはこのシートの実装の問題！ でした。修正済みです

In [ ]:
turn_list_of_list = [
    [
      '''以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。
### 指示:
ドラえもんの「のび太」になりきって会話を始めましょう。では以下の質問から始めてください：
"手を洗った後、エアドライヤーは必要だと思いますか？"
### 応答:''',
      '''### 指示:
町で一緒に食事をしましょう。バスで一緒に行きませんか？''',
    ],
    # [
    #     "複雑な方程式のセットから、各方程式からすべての変数名を抽出します。結果をJSON文字列として返し、各方程式に一行を割り当ててください。\n1) y = (3/4)x^3 - e^(2x) + sin(pi*x) - sqrt(7)\n2) 2A - B/(3+C) * sum(N=1 to 5; ln(N)^2) = 5D*integral(a=0 to pi; cos(comb(N=1 to 10; N*a)))\n3) E = m(c^2) + gamma*(v/d)/(-(alpha/2) + sqrt(beta^2 + (alpha/2)^2))",
    #     "方程式を並べ替えて、変数として 'a', 'b', 'c', 'd' などを使用してください。",
    # ],
    # [
    #     "以下の株価の記録から、2022年の各月の最高および最低の終値を抽出します。結果をCSV文字列として返し、各月に一行を割り当ててください。\n日付,開始,高値,安値,終値,ボリューム\n2022-01-01,150.02,155.28,148.50,153.80,15678900\n2022-01-02,154.32,157.25,153.48,156.25,19874500\n2022-02-01,160.50,163.28,159.50,161.80,14326700\n2022-02-02,161.80,164.25,161.30,163.90,17689200\n2022-03-01,165.40,168.35,163.10,166.80,16253400\n2022-03-02,167.00,169.85,165.50,168.20,19568100",
    #     "同じタスクをJSON形式で再度行い、回答内のすべての数値を最も近い整数に丸めてください。",
    # ]
]

In [ ]:
for turn_list in turn_list_of_list:
    prev_text = ""
    prev_output_text = ""
    for i, turn_text in enumerate(turn_list):
        if i != 0:
            # 直前のターンてinputと結果を加える
            turn_text = prev_text + '\n' + prev_output_text + '\n'+ turn_text

        print("input_text: ", turn_text)
        model.generation_config.pad_token_id = model.generation_config.eos_token_id
        inputs = tokenizer(turn_text, return_tensors="pt", add_special_tokens=add_special_tokens).to(model.device)
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, repetition_penalty=repetition_penalty, do_sample=do_sample, temperature=temperature)
        # outputsの先頭のtensorがinputsと同じ部分までは表示しない
        filtered_outputs = outputs[0][len(inputs.input_ids[0]):]
        filtered_output_text = tokenizer.decode(filtered_outputs, skip_special_tokens=skip_special_tokens)
        print("filtered_output_text: ", filtered_output_text)
        print("---------------------------------\n")

        # print("inputs: ", inputs.input_ids)
        # print("outputs: ", outputs[0])
        prev_text = turn_text
        prev_output_text = filtered_output_text

        output_text = tokenizer.decode(outputs[0], skip_special_tokens=skip_special_tokens)
        print("補足)output_text: ", output_text)

    print("filtered_output_text: ", prev_output_text)
    print("===================\n")

input_text:  以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。
### 指示:
ドラえもんの「のび太」になりきって会話を始めましょう。では以下の質問から始めてください：
"手を洗った後、エアドライヤーは必要だと思いますか？"
### 応答:
filtered_output_text:   "YES"</s>
---------------------------------

補足)output_text:  <s> 以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。
### 指示:
ドラえもんの「のび太」になりきって会話を始めましょう。では以下の質問から始めてください：
"手を洗った後、エアドライヤーは必要だと思いますか？"
### 応答: "YES"</s>
input_text:  以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。
### 指示:
ドラえもんの「のび太」になりきって会話を始めましょう。では以下の質問から始めてください：
"手を洗った後、エアドライヤーは必要だと思いますか？"
### 応答:
 "YES"</s>
### 指示:
町で一緒に食事をしましょう。バスで一緒に行きませんか？
filtered_output_text:   
"はい、喜んで。どこに行きますか？"</s>
---------------------------------

補足)output_text:  <s> 以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。
### 指示:
ドラえもんの「のび太」になりきって会話を始めましょう。では以下の質問から始めてください：
"手を洗った後、エアドライヤーは必要だと思いますか？"
### 応答:
 "YES"</s> 
### 指示:
町で一緒に食事をしましょう。バスで一緒に行きませんか？
"はい、喜んで。どこに行きますか？"</s>
filtered_output_text:   
"はい、喜んで。どこに行きますか？"</s>

